<a href="https://colab.research.google.com/github/ujkim25/sparta_ai_engineering/blob/main/Vector_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.2 MB/s eta 0:00:00


In [2]:
import requests
from io import StringIO
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

In [3]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

In [4]:
data = pd.read_csv(StringIO(res.text), sep="\t")
data.head(3)

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT


In [5]:
sentences = data["sentence_A"].tolist()
print(len(sentences))

4500


In [6]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [7]:
for url in urls:
    res = requests.get(url)
    data=pd.read_csv(StringIO(res.text), sep="\t", header=None, on_bad_lines="skip")
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [8]:
len(sentences)

15970

In [9]:
output_sentences=[]

for word in list(set(sentences)):
    if type(word) is str:
        output_sentences.append(word)

In [10]:
len(output_sentences)

12848

In [29]:
model = SentenceTransformer("bert-base-nli-mean-tokens")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
model.to("cuda")

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [31]:
sentence_embeddings = model.encode(output_sentences)

In [32]:
sentence_embeddings.shape

(12848, 768)

인스턴스가 12848개 있고 각 인스턴스는 768차원

In [33]:
sentence_embeddings[20]
# Dense Vector

array([-6.04027152e-01,  2.16417253e-01, -8.61041248e-01,  1.41237937e-02,
        2.54013121e-01,  2.35788152e-01,  8.31211269e-01, -3.96384835e-01,
       -4.61666942e-01, -4.94324684e-01, -3.75840634e-01, -7.54925668e-01,
        4.96960044e-01,  4.50599700e-01,  1.16344404e+00, -8.61196816e-02,
       -5.61471224e-01, -1.76719651e-01,  3.46563816e-01,  2.66958803e-01,
       -9.05754447e-01, -4.49605763e-01,  2.69876271e-01, -3.77578080e-01,
       -3.18889916e-01,  4.30738151e-01,  8.18732500e-01,  5.17193019e-01,
       -1.77960241e+00,  1.67231038e-01, -5.45678377e-01,  3.80041122e-01,
        4.84630466e-01,  5.49222715e-02,  5.71104176e-02,  6.08703554e-01,
       -9.04516399e-01, -3.21968347e-01, -1.11168317e-01, -1.84734738e+00,
       -8.63314122e-02,  7.24271059e-01, -9.19991076e-01, -2.62424424e-02,
       -1.00605106e+00, -2.72413701e-01, -1.51141596e+00,  5.50853252e-01,
       -9.73317564e-01,  7.24767745e-01, -1.38800770e-01,  7.40912855e-01,
       -1.70646048e+00, -

In [34]:
sentence_embeddings.device

'cpu'

In [35]:
dim = sentence_embeddings.shape[1]

In [36]:
dim

768

In [37]:
index = faiss.IndexFlatL2(dim)
index.is_trained

True

In [38]:
index.add(sentence_embeddings)

In [39]:
index.ntotal

12848

In [40]:
k = 4
xq = model.encode(["someone sprints with a football"])
xq.shape

(1, 768)

In [41]:
D, I = index.search (xq, k)

In [42]:
print(I)

[[11065  8910  7426    48]]


In [44]:
for idx in [11065,  8910,  7426,    48]:
    print(output_sentences[idx])

A group of football players is running in the field
A group of people playing football is running in the field
Two groups of people are playing football
A football player kicks the ball.


In [45]:
vec = index.reconstruct(11065)
vec.shape

(768,)

In [46]:
nlist=50
quantizer=faiss.IndexFlatL2(dim)
index_2=faiss.IndexIVFFlat(quantizer,dim,nlist)

In [47]:
index_2.is_trained

False

In [48]:
index_2.train(sentence_embeddings)

In [49]:
index_2.is_trained

True

In [50]:
index_2.add(sentence_embeddings)

In [51]:
index_2.ntotal

12848

In [53]:
D, I = index_2.search(xq,k)

In [54]:
print(I)

[[7426   48  849 1193]]


In [55]:
for idx in [7426,   48,  849, 1193]:
    print(output_sentences[idx])

Two groups of people are playing football
A football player kicks the ball.
A football player is running past an official carrying a football
Different teams are playing football on a field


In [ ]:
D, I = index.search (xq, k)

In [ ]:
D, I = index_2.search(xq,k)

In [56]:
!pip install -qU \
    pinecone-datasets \
    pinecone \
    pinecone-text \
    torch \
    datasets \
    transformers \
    sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 779.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall pinecone-client -y

In [57]:
import os
from pinecone import Pinecone

In [58]:
api_key = "pcsk_2QAWHn_BUHoWehZ8rNth15NPtnDDrcnxDe3umXtHVeWjYRuvg5KF8ELw7oSeeNd4qrdkAY"

In [59]:
pc = Pinecone(api_key=api_key)

In [60]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(cloud = "aws", region = "us-east-1")

In [61]:
index_name = "test"

In [63]:
import time

if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=384,
        metric="dotproduct",
        spec=spec
    )

In [65]:
pc.list_indexes().names()

['test']

In [66]:
index = pc.Index(index_name)

In [68]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [69]:
from datasets import load_dataset

# load the medical dataset from huggingfac dataset hub
pubmed = load_dataset('pubmed_qa', 'pqa_labeled', split='train')

contexts = []
# loop through the context passages
for record in pubmed['context']:
    # join context passages for each question and append to contexts list
    contexts.append('\n'.join(record['contexts']))
# view some of the contexts
for context in contexts[:2]:
    print(f"{context[:300]}...")

README.md: 0.00B [00:00, ?B/s]

pqa_labeled/train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cel...
Assessment of visual acuity depends on the optotypes used for measurement. The ability to recognize different optotypes differs even if their critical details appear under the same visual angle. Since optotypes are evaluated on individuals with good visual acuity and without eye disorders, differenc...


In [70]:
model = SentenceTransformer(
    'multi-qa-MiniLM-L6-cos-v1',
    device='cuda'
)
model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [75]:
for i in range(0,len(contexts[:300])):
    context = contexts[i]
    id = str(i)

    meta = {"context":context}

    dense_embed = model.encode([context]).tolist()[0]

    index.upsert(
        [
            {
                "id":id,
                "values":dense_embed,
                "metadata":meta
            }
        ]
    )

In [76]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {'': {'vector_count': 300}},
 'total_vector_count': 300,
 'vector_type': 'dense'}

In [77]:
question = "Can clinicians use the PHQ-9 to assess depression in people with vision loss?"

In [78]:
dense_vec = model.encode(question).tolist()

In [79]:
len(dense_vec)

384

In [81]:
result = index.query(
    vector=dense_vec,
    top_k=3,
    include_metadata=True
)

In [82]:
result["matches"][1]

{'id': '125',
 'metadata': {'context': 'There is increasing pressure on mental health '
                         'providers to reduce the duration of treatments, '
                         'while retaining level of quality and effectiveness. '
                         'The risk is that the population is underserved and '
                         'therefore needs new treatment episodes. The primary '
                         'aim of this study was to investigate whether '
                         'duration of treatment and return into mental health '
                         'care were related.\n'
                         'This study examined Dutch patients with an initial '
                         'treatment episode in 2009 or 2010 in specialized '
                         'mental health settings for depressive disorder (N\xa0'
                         '=\xa085,754). Follow-up data about treatment '
                         'episodes were available up until 2013. The data set '
      